In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import csv
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk

In [0]:
import warnings
warnings.filterwarnings('ignore')

In [0]:
data= pd.read_csv(r'/content/drive/My Drive/amazon_books_review.csv')

In [0]:
df_cleaned = data.head(100000)
df_cleaned['star_rating'].count()
df_cleaned['star_rating'].unique()
df_cleaned['star_rating'].value_counts()
indexNames = df_cleaned[ df_cleaned['star_rating'] == 'star_rating' ].index
df_cleaned.drop(indexNames,inplace=True)
df_cleaned.dropna(inplace=True)

In [0]:
df_cleaned['star_rating']=df_cleaned['star_rating'].astype(float).astype(int)
df_cleaned['review_body']=df_cleaned['review_body'].str.replace('<br />', r'')
df_cleaned['review_body']=df_cleaned['review_body'].str.replace("[,'!;.–`:?><-]",'')
#df_cleaned['review_body']=df_cleaned['review_body'].str.replace("[!#$%&'()*+, -./:;<=>?@[\]^_`{|}~]",'')
df_cleaned['review_body']=df_cleaned['review_body'].str.lower()
#df_cleaned=df_cleaned[df_cleaned.review_body.str.split(' ').str.len()<300]
df_cleaned['review_body']=df_cleaned.review_body.str.replace('[^a-zA-Z ]', '')
df_cleaned.dropna(inplace=True)
df_cleaned.isna().sum()

star_rating        0
review_headline    0
review_body        0
dtype: int64

In [0]:
def partition(x):
    if x<3:
        return 'Negative'
    elif x>3:
        return 'Positive'
    return 'Neutral'

actualScore = df_cleaned['star_rating']
positiveNegative = actualScore.map(partition)
df_cleaned['star_rating'] = positiveNegative

In [0]:
X_train=df_cleaned['review_body']
y_train=df_cleaned['star_rating']

In [0]:
data_test= pd.read_csv(r'/content/drive/My Drive/test.csv')
df_cleaned = data_test
indexNames = df_cleaned[ df_cleaned['ratings'] == 'ratings' ].index
df_cleaned.drop(indexNames,inplace=True)
df_cleaned.dropna(inplace=True)

In [0]:
df_cleaned['review_body']=df_cleaned['review_body'].str.replace('<br />', r'')
df_cleaned['review_body']=df_cleaned['review_body'].str.replace("[,'!;.–`:?><-]",'')
#df_cleaned['review_body']=df_cleaned['review_body'].str.replace("[!#$%&'()*+, -./:;<=>?@[\]^_`{|}~]",'')
df_cleaned['review_body']=df_cleaned['review_body'].str.lower()
#df_cleaned=df_cleaned[df_cleaned.review_body.str.split(' ').str.len()<300]
df_cleaned['review_body']=df_cleaned.review_body.str.replace('[^a-zA-Z ]', '')
df_cleaned.dropna(inplace=True)
df_cleaned.isna().sum()
X_test=df_cleaned['review_body']
y_test=df_cleaned['ratings']

In [0]:
clf = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', LogisticRegression(max_iter=1000)),
])
clf = clf.fit(X_train, y_train)
accuracy_train=accuracy_score(y_train, clf.predict(X_train))
accuracy_test=accuracy_score(y_test, clf.predict(X_test))
print('Accuracy on train %f' %accuracy_train)
print('Accuracy on test %f' %accuracy_test)

Accuracy on train 0.979218
Accuracy on test 0.835988


In [0]:
clf = Pipeline([
    ('vect', CountVectorizer(analyzer='word', ngram_range=(1, 2))),
    ('clf', LogisticRegression(max_iter=500,C=0.1,random_state=40,solver='newton-cg')),
])
clf = clf.fit(X_train, y_train)
accuracy_train=accuracy_score(y_train, clf.predict(X_train))
accuracy_test=accuracy_score(y_test, clf.predict(X_test))
print('Accuracy on train %f' %accuracy_train)
print('Accuracy on test %f' %accuracy_test)

Accuracy on train 0.990740
Accuracy on test 0.864429


In [0]:
clf = Pipeline([
    ('vect', TfidfVectorizer(analyzer='word', ngram_range=(1, 2))),
    ('clf', LogisticRegression(max_iter=500,C=0.1,random_state=40,solver='newton-cg')),
])
clf = clf.fit(X_train, y_train)
accuracy_train=accuracy_score(y_train, clf.predict(X_train))
accuracy_test=accuracy_score(y_test, clf.predict(X_test))
print('Accuracy on train %f' %accuracy_train)
print('Accuracy on test %f' %accuracy_test)

Accuracy on train 0.811869
Accuracy on test 0.805868


In [0]:
from sklearn.tree import DecisionTreeClassifier
clf = Pipeline([
    ('vect', CountVectorizer(analyzer='word', ngram_range=(1, 2))),
    ('clf', DecisionTreeClassifier()),
])
clf = clf.fit(X_train, y_train)
accuracy_train=accuracy_score(y_train, clf.predict(X_train))
accuracy_test=accuracy_score(y_test, clf.predict(X_test))
print('Accuracy on train %f' %accuracy_train)
print('Accuracy on test %f' %accuracy_test)

Accuracy on train 0.999920
Accuracy on test 0.760838


In [0]:
from sklearn.tree import DecisionTreeClassifier
clf = Pipeline([
    ('vect',TfidfVectorizer(analyzer='word', ngram_range=(1, 2))),
    ('clf', DecisionTreeClassifier()),
])
clf = clf.fit(X_train, y_train)
accuracy_train=accuracy_score(y_train, clf.predict(X_train))
accuracy_test=accuracy_score(y_test, clf.predict(X_test))
print('Accuracy on train %f' %accuracy_train)
print('Accuracy on test %f' %accuracy_test)

Accuracy on train 0.999920
Accuracy on test 0.739717


In [0]:
from sklearn.tree import DecisionTreeClassifier
clf = Pipeline([
    ('vect',TfidfVectorizer()),
    ('clf', DecisionTreeClassifier()),
])
clf = clf.fit(X_train, y_train)
accuracy_train=accuracy_score(y_train, clf.predict(X_train))
accuracy_test=accuracy_score(y_test, clf.predict(X_test))
print('Accuracy on train %f' %accuracy_train)
print('Accuracy on test %f' %accuracy_test)

In [0]:
from sklearn.tree import DecisionTreeClassifier
clf = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', DecisionTreeClassifier()),
])
clf = clf.fit(X_train, y_train)
accuracy_train=accuracy_score(y_train, clf.predict(X_train))
accuracy_test=accuracy_score(y_test, clf.predict(X_test))
print('Accuracy on train %f' %accuracy_train)
print('Accuracy on test %f' %accuracy_test)

Accuracy on train 0.999920
Accuracy on test 0.750318


In [0]:
from sklearn.tree import DecisionTreeClassifier
clf = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', DecisionTreeClassifier(criterion= "entropy")),
])
clf = clf.fit(X_train, y_train)
accuracy_train=accuracy_score(y_train, clf.predict(X_train))
accuracy_test=accuracy_score(y_test, clf.predict(X_test))
print('Accuracy on train %f' %accuracy_train)
print('Accuracy on test %f' %accuracy_test)

Accuracy on train 0.999920
Accuracy on test 0.738537


In [0]:
from sklearn.svm import LinearSVC
clf = Pipeline([
    ('vect', CountVectorizer()),
    ('clf',LinearSVC()),
])
clf = clf.fit(X_train, y_train)
accuracy_train=accuracy_score(y_train, clf.predict(X_train))
accuracy_test=accuracy_score(y_test, clf.predict(X_test))
print('Accuracy on train %f' %accuracy_train)
print('Accuracy on test %f' %accuracy_test)

Accuracy on train 0.993964
Accuracy on test 0.815388


In [0]:
from sklearn.svm import LinearSVC
clf = Pipeline([
    ('vect',TfidfVectorizer()),
    ('clf',LinearSVC()),
])
clf = clf.fit(X_train, y_train)
accuracy_train=accuracy_score(y_train, clf.predict(X_train))
accuracy_test=accuracy_score(y_test, clf.predict(X_test))
print('Accuracy on train %f' %accuracy_train)
print('Accuracy on test %f' %accuracy_test)

Accuracy on train 0.956664
Accuracy on test 0.851949
